In [1]:
print("Hello World")

Hello World


In [ ]:
%pip install openai 
%pip install python-dotenv

In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv

Save the OPENAI_KEY into ".env" file, and *** AND .env TO YOUR .gitignore FILE. ***

In [6]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

For normal use GPT-4 test

In [ ]:
MODEL = "gpt-4"
# MODEL = "gpt-3.5-turbo"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "assistant",
            "content": "You are a meeting organzer. Help me generate 20 meeting IDs. \n Here is the example: \n ```mambf90j96gb0m3p8u9fo0q7v8```",
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

Prompt for calendar questions

In [48]:
# prompt = """ You will be provided with a dataframe "calendar_data". "calendar_data" consists of 7 columns, which can be represented as calendar_Data(ID, status, summary, start, end , duration, attendees).
# Input dataset: ID is the meeting ID; start is the meeting start time; end is the meeting end time; duration is how long the meeting last; attendees is who will attend the meeting

# Write 10 different questions to operate dataframe. For example:

# "create meeting with Natalia Shreve tomorrow at 10 am",
# "remove tomorrow's meeting at 10 am",
# "reschedule recurrent daily Meeting with Bob to 5 pm".
# "Do I have meeting today"
# """

prompt = f""" You are provided a ```dataframe(ID, status, summary, start, end , duration, attendees)```. This dataframe includes all you calenders.
Your task is generate questions to query this dataframe. Output a json object that contains the following keys: question, answer. Fill the answer key blank.

Output JSON: <json with question and answer>
"""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

{
"question": "What is the status and summary of the event with ID 12345?",
"answer": ""
}

{
"question": "How many attendees are there for the meeting with ID 23456?",
"answer": ""
}

{
"question": "What is the duration of the event with ID 34567?",
"answer": ""
}

{
"question": "When does the event with ID 45678 start and end?",
"answer": ""
}

{
"question": "What is the longest event on my calendar?",
"answer": ""
}

{
"question": "Who are the attendees for the event with ID 56789?",
"answer": ""
}

{
"question": "Find events with the status 'Cancelled'?",
"answer": ""
}

{
"question": "What events are scheduled to start tomorrow?",
"answer": ""
}

{
"question": "List events with duration longer than 2 hours?",
"answer": ""
}

{
"question": "How many events do I have scheduled for today?",
"answer": ""
}


In [ ]:
# convert output into json

Prompt for generting meeting date with format YYYY-MM-DD hh:mm:ss-00:00

In [45]:
prompt = """ Help me generate 20 meeting date, including start date and end date. 
Date format: YYYY-MM-DD hh:mm:ss-00:00

Limitations:
The meeting usually lasts 30 minutes to 2 hours;
The meeting should start from 2024-02-03 to 2024-04-05
Every tuesday, there are at least 2 meetings


Here is the example: 
{Start: 2024-03-24 10:00:00-00:00
 End:   2024-03-24 10:45:00-00:00   
}

Results:
"""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

1. {Start: 2024-02-06 09:00:00-00:00, End: 2024-02-06 10:30:00-00:00}
2. {Start: 2024-02-06 14:00:00-00:00, End: 2024-02-06 15:30:00-00:00}
3. {Start: 2024-02-13 09:00:00-00:00, End: 2024-02-13 10:30:00-00:00}
4. {Start: 2024-02-13 14:00:00-00:00, End: 2024-02-13 15:30:00-00:00}
5. {Start: 2024-02-20 09:30:00-00:00, End: 2024-02-20 11:00:00-00:00}
6. {Start: 2024-02-20 14:00:00-00:00, End: 2024-02-20 15:30:00-00:00}
7. {Start: 2024-02-27 09:00:00-00:00, End: 2024-02-27 10:30:00-00:00}
8. {Start: 2024-02-27 14:00:00-00:00, End: 2024-02-27 15:30:00-00:00}
9. {Start: 2024-03-05 09:30:00-00:00, End: 2024-03-05 11:00:00-00:00}
10. {Start: 2024-03-05 14:00:00-00:00, End: 2024-03-05 15:30:00-00:00}
11. {Start: 2024-03-12 09:00:00-00:00, End: 2024-03-12 10:30:00-00:00}
12. {Start: 2024-03-12 14:00:00-00:00, End: 2024-03-12 15:30:00-00:00}
13. {Start: 2024-03-19 09:30:00-00:00, End: 2024-03-19 11:00:00-00:00}
14. {Start: 2024-03-19 14:00:00-00:00, End: 2024-03-19 15:30:00-00:00}
15. {Start: 202

Prompt for generting meeting topic and attendees

In [43]:
prompt = """ You are walter, a new software engineer employee of NinjaTech AI. You have many meetings with team members, 
for example  meeting with the team leader Arash about the project progress and research plan. You have 4 team members: Alice, Bob, Carrie, David. You have regular meeting every Monday about project progress.
You also need to talk to them about research issues. 
Help me generate all your 10 meetings, in terms of meeting topic and attendees. 
Meeting attendees are their ninjatech emails, for exampole ${name}$@ninjatech.ai(${status}$).  
${name}$ is their name and ${status}$ denotes if they accepted the meeting invitation. ${status}$ could be accepted, needsAction, and rejected.
Each meeting usually include 2 people but no more than the all people in this company.
The output format is ${name}$@ninjatech.ai(${status}$)\n ${name}$@ninjatech.ai(${status}$)
Here is the example: 
"meeting Topic: 
temp-testing@ninjatech.ai(accepted)\n yevhenia.spiridonova@ninjatech.ai(needsAction)"
"""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

Sure, let's generate these 10 meetings:

1. Meeting Topic: Project Kickoff 
walter@ninjatech.ai(accepted)
 arash@ninjatech.ai(accepted)

2. Meeting Topic: Agile Development Workflow
walter@ninjatech.ai(accepted)
 alice@ninjatech.ai(accepted)

3. Meeting Topic: Code Review 
walter@ninjatech.ai(accepted)
 bob@ninjatech.ai(accepted)

4. Meeting Topic: Research Planning
walter@ninjatech.ai(accepted)
 carrie@ninjatech.ai(accepted)

5. Meeting Topic: Debugging And Error Handling 
walter@ninjatech.ai(accepted)
 david@ninjatech.ai(accepted)

6. Meeting Topic: Weekly Progress Review
walter@ninjatech.ai(accepted)
 arash@ninjatech.ai(needsAction)

7. Meeting Topic: Database Architecture 
walter@ninjatech.ai(accepted)
 alice@ninjatech.ai(needsAction)

8. Meeting Topic: Security Best Practices 
walter@ninjatech.ai(accepted)
 bob@ninjatech.ai(accepted)

9. Meeting Topic: UI/UX Design 
walter@ninjatech.ai(accepted)
 carrie@ninjatech.ai(rejected)

10. Meeting Topic: API Integration 
walter@ninjatech.a

In [ ]:
prompt = """ You are walter, a new software engineer employee of NinjaTech AI. You have many meetings team members, 
for example meet with organizer Natalia about the job application, interview process, contract discussion and meeting with the team leader Arash. 
Help me generate all your 10 meetings, in terms of meeting topic and attendees. 
Meeting attendees are their ninjatech emails, for exampole ${name}$@ninjatech.ai(${status}$).  
${name}$ is their name and ${status}$ denotes if they accepted the meeting invitation. ${status}$ could be accepted, needsAction, and rejected.
Each meeting usually include 2 people but no more than the all people in this company.
The output format is ${name}$@ninjatech.ai(${status}$)\n ${name}$@ninjatech.ai(${status}$)
Here is the example: 
"meeting Topic: 
temp-testing@ninjatech.ai(accepted)\n yevhenia.spiridonova@ninjatech.ai(needsAction)"
"""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

Prompt for generating attendees

In [36]:
prompt = """ You are a meeting organzer. Help me generate 7 meeting start date and end data for this week. Today is 2024-03-27 Wednesday.
The meeting may be in several days age, may be in today, and may be several days late. For each day, there are no meeting, 1 meeting or 2 meeting.
The meeting usually lasts for 10 minutes to 2 hours.
Date format: YYYY-MM-DD hh:mm:ss-00:00. The meeting should be in time order
Here is the example: 
{Start date: 2024-03-27 12:00:00-00:00
 end date: 2024-03-27 12:30:00-00:00
}
"""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

Sure, you can use the `json` module provided by Python to achieve this. Below is a simple function that converts a multiline string into a .json format:

```python
import json

def convert_to_json(multiline_string):
    # Split the multiline string into a list of lines
    lines = multiline_string.split("\n")
    
    # Convert the list into a json string
    json_string = json.dumps(lines)
    
    return json_string

# Test the function
multiline_string = """
This is line 1
This is line 2
This is line 3
"""

print(convert_to_json(multiline_string))
```

Please note that this code would convert each line in the multiline string as a separate item of an array in the resulting .json.

If your multiline string is actually representing a .json, you don't need to split your string line by line. You can directly load it into .json as shown below:

```python
import json

def convert_to_json(multiline_json_string):
    try:
        json_data = json.loads(multiline_json_string)
        return 

Verify the ***gpt-3.5-turbo***

In [12]:
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    # {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    # {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    {"role": "user", "content": "Are you GPT4 or GPT3.5?"}
  ]
)

print(completion.choices[0].message.content)

I am an instance of OpenAI's GPT-3 model.
